BLmodel

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Load data files
predictions_2028_path = '2028_predictions.csv'
actual_results_path = '2024_actual.csv'
program_data_path = 'clean_program_data.csv'
athletes_data_path = 'clean_athletes_data.csv'

# Read CSV files
predictions_2028 = pd.read_csv(predictions_2028_path)
actual_results = pd.read_csv(actual_results_path)
program_data = pd.read_csv(program_data_path)
athletes_data = pd.read_csv(athletes_data_path)

# Filter data
actual_results = actual_results[actual_results["Year"] == 2024]
athletes_data = athletes_data[athletes_data["Year"] == 2024]
unique_names_per_noc = athletes_data.groupby('NOC').agg({
    'Name': 'nunique'
}).reset_index().rename(columns={'Name': 'Unique_Names'})
# Merge and preprocess data
data2428 = pd.merge(
    predictions_2028.rename(columns={"Predicted_Score": "Score_2028"}),
    actual_results[["NOC", "Total_Final_Score"]].rename(columns={"Total_Final_Score": "Score_2024"}),
    on="NOC",
    how="left"
)
data2428 = pd.merge(
    data2428,
    unique_names_per_noc,
    on="NOC",
    how="left"
)
# Normalize Innovation Coefficient
scaler = MinMaxScaler()
data2428["Innovation_Coeff"] = scaler.fit_transform(
    (data2428["Score_2028"] - data2428["Score_2024"]).fillna(0).values.reshape(-1, 1)
)

data2428["Team_Avg"] = data2428["Score_2024"] / data2428["Unique_Names"]
# Extract the team averages
team_averages = data2428['Team_Avg'].values

# Calculate the absolute difference matrix
difference_matrix = np.abs(team_averages[:, None] - team_averages)

# Invert the difference matrix, avoiding division by zero by adding a small constant
inverse_difference_matrix = 1 / (difference_matrix + 1e-6)

# Standardize the inverse difference matrix
scaler = StandardScaler()
standardized_matrix = scaler.fit_transform(inverse_difference_matrix)
np.fill_diagonal(standardized_matrix, 0)

# Create a DataFrame for better visualization
competition_matrix = pd.DataFrame(
    standardized_matrix, 
    index=data2428['NOC'], 
    columns=data2428['NOC']
)


In [29]:
from scipy.integrate import solve_ivp

# Bass-Volterra model
def bass_volterra(t, R, p, alpha):
    dRdt = []
    for i in range(len(R)):
        innovation = p[i] * (1 - R[i])
        competition = sum(alpha[i][j] * R[i] * R[j] for j in range(len(R)) if j != i)
        dRdt.append(innovation - competition)
    return dRdt

# Medal count simulation
def simulate_medal_count(data, n_ticks, total_gold, total_silver, total_bronze):
    results = []
    remaining_data = data.copy()  # Copy data for iterative modification
    medal_limits = {"Gold": total_gold, "Silver": total_silver, "Bronze": total_bronze}

    for medal_type, total_medals in medal_limits.items():
        if "Initial_Medal_Count" not in remaining_data:
            remaining_data["Initial_Medal_Count"] = np.zeros(len(remaining_data))
        current_indices = remaining_data.index
        R = remaining_data["Initial_Medal_Count"].values
        p = remaining_data["Innovation_Coeff"].values
        
        # typeerror here
        alpha = competition_matrix[np.ix_(current_indices, current_indices)]

        solution = solve_ivp(bass_volterra, (0, n_ticks), R, t_eval=np.arange(0, n_ticks, 1), args=(p, alpha))
        remaining_data[medal_type] = solution.y[:, -1]
        
        remaining_data = remaining_data.sort_values(by=medal_type, ascending=False)
        awarded = remaining_data.head(total_medals).copy()
        awarded[medal_type] = 1
        
        remaining_data = remaining_data.iloc[total_medals:].copy()
        results.append(awarded[["NOC", medal_type]].copy())
    return pd.concat(results, axis=0)

# Simulation parameters
n_ticks = program_data.shape[0]
total_gold = 339
total_silver = 338
total_bronze = 395

# Run simulation
final_results = simulate_medal_count(data2428, n_ticks, total_gold, total_silver, total_bronze)

# Output results
gold_count = final_results[final_results["Gold"] == 1].shape[0] if "Gold" in final_results else 0
silver_count = final_results[final_results["Silver"] == 1].shape[0] if "Silver" in final_results else 0
bronze_count = final_results[final_results["Bronze"] == 1].shape[0] if "Bronze" in final_results else 0

print(f"Gold: {gold_count}, Silver: {silver_count}, Bronze: {bronze_count}")

RangeIndex(start=0, stop=205, step=1)


InvalidIndexError: (array([[  0],
       [  1],
       [  2],
       [  3],
       [  4],
       [  5],
       [  6],
       [  7],
       [  8],
       [  9],
       [ 10],
       [ 11],
       [ 12],
       [ 13],
       [ 14],
       [ 15],
       [ 16],
       [ 17],
       [ 18],
       [ 19],
       [ 20],
       [ 21],
       [ 22],
       [ 23],
       [ 24],
       [ 25],
       [ 26],
       [ 27],
       [ 28],
       [ 29],
       [ 30],
       [ 31],
       [ 32],
       [ 33],
       [ 34],
       [ 35],
       [ 36],
       [ 37],
       [ 38],
       [ 39],
       [ 40],
       [ 41],
       [ 42],
       [ 43],
       [ 44],
       [ 45],
       [ 46],
       [ 47],
       [ 48],
       [ 49],
       [ 50],
       [ 51],
       [ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [ 58],
       [ 59],
       [ 60],
       [ 61],
       [ 62],
       [ 63],
       [ 64],
       [ 65],
       [ 66],
       [ 67],
       [ 68],
       [ 69],


In [30]:

from scipy.integrate import solve_ivp

# Bass-Volterra model
def bass_volterra(t, R, p, alpha):
    dRdt = []
    for i in range(len(R)):
        innovation = p[i] * (1 - R[i])
        competition = sum(alpha[i][j] * R[i] * R[j] for j in range(len(R)) if j != i)
        dRdt.append(innovation - competition)
    return dRdt
def simulate_medal_count(data, n_ticks, total_gold, total_silver, total_bronze):
    results = []
    remaining_data = data.copy()  # Copy data for iterative modification
    medal_limits = {"Gold": total_gold, "Silver": total_silver, "Bronze": total_bronze}

    for medal_type, total_medals in medal_limits.items():
        if "Initial_Medal_Count" not in remaining_data:
            remaining_data["Initial_Medal_Count"] = np.zeros(len(remaining_data))
        current_indices = remaining_data.index
        R = remaining_data["Initial_Medal_Count"].values
        p = remaining_data["Innovation_Coeff"].values
        
        # Corrected indexing here
        alpha = competition_matrix.loc[current_indices, current_indices].values  # Using .loc to ensure valid indexing

        solution = solve_ivp(bass_volterra, (0, n_ticks), R, t_eval=np.arange(0, n_ticks, 1), args=(p, alpha))
        remaining_data[medal_type] = solution.y[:, -1]
        
        remaining_data = remaining_data.sort_values(by=medal_type, ascending=False)
        awarded = remaining_data.head(total_medals).copy()
        awarded[medal_type] = 1
        
        remaining_data = remaining_data.iloc[total_medals:].copy()
        results.append(awarded[["NOC", medal_type]].copy())
    return pd.concat(results, axis=0)

# Simulation parameters
n_ticks = program_data.shape[0]
total_gold = 339
total_silver = 338
total_bronze = 395

# Run simulation
final_results = simulate_medal_count(data2428, n_ticks, total_gold, total_silver, total_bronze)

# Output results
gold_count = final_results[final_results["Gold"] == 1].shape[0] if "Gold" in final_results else 0
silver_count = final_results[final_results["Silver"] == 1].shape[0] if "Silver" in final_results else 0
bronze_count = final_results[final_results["Bronze"] == 1].shape[0] if "Bronze" in final_results else 0

print(f"Gold: {gold_count}, Silver: {silver_count}, Bronze: {bronze_count}")

KeyError: 'None of [RangeIndex(start=0, stop=205, step=1)] are in the [index]'